In [ ]:
import json
import csv
def json_to_csv(json_file_path, csv_file_path):
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)
    with open(csv_file_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['filename', 'label'])
        for filename, details in data.items():
            label = 0 if details['label'] == 'FAKE' else 1
            writer.writerow([filename, label])
json_file_path = 'data.json'
csv_file_path = 'data.csv'
json_to_csv(json_file_path, csv_file_path)

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
video_dir = 'E:\\ML\\deepfake-detection-challenge\\train_sample_videos'
csv_file = 'E:\\ML\\deepfake-detection-challenge\\sample_submission.csv'
df = pd.read_csv(csv_file)
img_size = 128 
frames_per_video = 10
def extract_frames(video_path, max_frames):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_skip = max(1, total_frames // max_frames)

    for i in range(max_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_skip)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (img_size, img_size))
            frames.append(frame)
        else:
            break
    cap.release()
    return frames
data = []
labels = []
for idx, row in df.iterrows():
    video_path = os.path.join(video_dir, row['filename'])
    label = row['label']
    frames = extract_frames(video_path, frames_per_video)
    for frame in frames:
        data.append(frame)
        labels.append(label)
data = np.array(data)
labels = to_categorical(labels, num_classes=2)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
X_train = X_train / 255.0
X_test = X_test / 255.0



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy:.4f}')

In [ ]:
model.save('deepfake_detection_model.h5')

In [6]:
!python -m pip install tensorflow==2.10.1

   ---------------------------------------- 0.0/455.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/455.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/455.9 MB 660.6 kB/s eta 0:11:31
   ---------------------------------------- 0.0/455.9 MB 660.6 kB/s eta 0:11:31
   ---------------------------------------- 0.1/455.9 MB 563.7 kB/s eta 0:13:29
   ---------------------------------------- 0.1/455.9 MB 714.4 kB/s eta 0:10:39
   ---------------------------------------- 0.2/455.9 MB 876.1 kB/s eta 0:08:41
   ---------------------------------------- 0.2/455.9 MB 656.9 kB/s eta 0:11:34
   ---------------------------------------- 0.2/455.9 MB 758.5 kB/s eta 0:10:01
   ---------------------------------------- 0.3/455.9 MB 803.7 kB/s eta 0:09:27
   ---------------------------------------- 0.3/455.9 MB 803.7 kB/s eta 0:09:27
   ---------------------------------------- 0.3/455.9 MB 803.7 kB/s eta 0:09:27
   ---------------------------------------- 0.3/455.9 MB 5

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.36.0 requires protobuf<6,>=3.20, but you have protobuf 3.19.6 which is incompatible.


In [4]:
import cv2
import numpy as np
import tensorflow as tf
img_size = 128  
frames_per_video = 25  
model = tf.keras.models.load_model('deepfake_detection_model.h5')
def extract_frames(video_path, max_frames):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_skip = max(1, total_frames // max_frames)

    for i in range(max_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_skip)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (img_size, img_size))
            frames.append(frame)
        else:
            break
    cap.release()
    return frames

def predict_video(video_path):
    frames = extract_frames(video_path, frames_per_video)
    frames = np.array(frames) / 255.0 
    predictions = model.predict(frames)
    
    avg_prediction = np.mean(predictions, axis=0)
    final_prediction = np.argmax(avg_prediction)
    return final_prediction, avg_prediction

def prediction_to_label(prediction):
    return 'Fake' if prediction == 0 else 'Real'

video_path = 'C:\\Users\\achal\\Downloads\\1.jpg'
final_prediction, avg_prediction = predict_video(video_path)
label = prediction_to_label(final_prediction)

print(f'The video is predicted to be: {label}')
print(f'Confidence scores (Fake, Real): {avg_prediction}')


1/1 [==============================] - 0s 44ms/step
The video is predicted to be: Real
Confidence scores (Fake, Real): [2.6134541e-07 9.9999976e-01]
